In [1]:
# import pymongo
import requests
import pandas as pd
import time


API_KEY = 'RGAPI-c7cfbbf1-4fd8-46a8-acd7-418cc544c57d'
games = []

In [2]:
def get_account_id(playerName):
    response = requests.get('https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'\
                            + playerName + '?api_key=' + API_KEY)
    json = response.json()
    accountId = json['accountId']
    return accountId



In [3]:
def get_game_ids(accId, copy_game_ids):
    
    game_ids = []
    response1 = requests.get('https://br1.api.riotgames.com/lol/match/v4/matchlists/by-account/' + accId + '?beginIndex=0&endIndex=10&queue=420&api_key=' + API_KEY)
    json1 = response1.json()['matches']

    for game in json1:

        if not game['gameId'] in copy_game_ids:
            game_ids.append(str(game['gameId']))
            
    #game_ids[] stores the last 10 gameId of the selected player
    return game_ids

In [4]:
def get_games(gIds):
    '''
    Function that get x games 
    
    parameters: 
    ----------
        gIds: list or string containing gameIds
    
    returns:
    ---------
        games: list of dicts containing each game infos
    '''
    if isinstance(gIds, str):
        gIds = [gIds]
    
    for game_id in gIds:
        
        response2 = requests.get('https://br1.api.riotgames.com/lol/match/v4/matches/'\
                                 + game_id + '?api_key=' + API_KEY)
        json = response2.json()
        #then for each 'game_id', request the info of the match 

        dict_game = {}
        dict_game['game_id'] = game_id
        #create a dict that will store the info of the current match

        for n_team in range(2):
            for n_ban in range(5):
                ban = json['teams'][n_team]['bans'][n_ban]['championId']
                #Ban ['teams'] 0 to 1 / 0 to 4
                dict_game['ban_team_{}_{}'.format(n_team, n_ban)] = ban
                #put the 5 bans of each team 

            t_dragon = json['teams'][n_team]['dragonKills']
            dict_game['t_dragon_{}'.format(n_team)] = t_dragon

            t_herald = json['teams'][n_team]['riftHeraldKills']
            dict_game['t_herald_{}'.format(n_team)] = t_herald

            t_turrets = json['teams'][n_team]['towerKills']
            dict_game['t_turrets_{}'.format(n_team)] = t_turrets

            t_barons = json['teams'][n_team]['baronKills']
            dict_game['t_barons_{}'.format(n_team)] = t_barons

            have_team_win = json['teams'][n_team]['win']
            dict_game['have_team_win_{}'.format(n_team)] = have_team_win
            #put the data inside the dict. range 2 bcz theres only 2 teams

        for n_part in range(10):
            p_identity = json['participantIdentities'][n_part]['player']['accountId']
            dict_game['p_identity_{}'.format(n_part)] = p_identity        

            p_kills = json['participants'][n_part]['stats']['kills']
            dict_game['p_kills_{}'.format(n_part)] = p_kills

            p_deaths = json['participants'][0]['stats']['deaths']
            dict_game['p_deaths_{}'.format(n_part)] = p_deaths

            p_assists = json['participants'][0]['stats']['assists']
            dict_game['p_assists_{}'.format(n_part)] = p_assists

            p_gold = json['participants'][0]['stats']['goldEarned']
            dict_game['p_gold_{}'.format(n_part)] = p_gold

            p_vision_score = json['participants'][0]['stats']['visionScore']
            dict_game['p_vision_score_{}'.format(n_part)] = p_vision_score

            p_farm = json['participants'][0]['stats']['totalMinionsKilled']\
            + json['participants'][0]['stats']['neutralMinionsKilled'] 
            dict_game['p_farm_{}'.format(n_part)] = p_farm

            p_champ_id = json['participants'][0]['championId']
            dict_game['p_champ_id_{}'.format(n_part)] = p_champ_id

            p_team = json['participants'][0]['teamId']
            dict_game['p_team_{}'.format(n_part)] = p_team
            #add the info of each player of match on dict



        games.append(dict_game)
        #add the game at the dict of games
        time.sleep(2)

    return games
        

In [5]:
def save_games(games):
    '''
    function that saves the dict of games on a .json located on directory /games
    
    parameters:
    -----------
        games: dict containing the info of the games
    
    returns:
    --------
    player_ids: tuple with player_ids, copy_game_ids
    '''
    
    df = pd.DataFrame(games)
    
    
    player_ids = []
    copy_game_ids = []
    for p in df.columns:
        if p.startswith('p_identity'):
            player_ids += df[p].tolist()
        
        if p == 'game_id':
            copy_game_ids += df[p].tolist()
        

    
    df.to_json('games/partidas_{}.json'.format(round(time.time())))
    return set(player_ids), set(copy_game_ids)

In [6]:
round(time.time())

1550185059

In [7]:
#player_ids = []
#copy_game_ids = []
#for p in df.columns:
#    if p.startswith('p_identity'):
#        player_ids += df[p].tolist()
#        
#    if p == 'game_id':
#        copy_game_ids += df[p].tolist()        
    


In [8]:
player_name = 'chaoshimself'
copy_game_ids = []

accId = get_account_id(player_name)

player_ids = [accId]

for n in range(2):
    print(len(player_ids))
    for player_id in player_ids:
        print(player_ids, copy_game_ids)

        game_ids = get_game_ids(player_id, copy_game_ids)
    #         print(game_ids)

        games = get_games(game_ids)
    #         print(games)

        player_ids, copy = save_games(games)
#         print(player_ids, copy)
#         copy_game_ids.append(copy)
        
    
#     print(n)
        

1
['2iOOSqQD5fvpgiS7I_DpV_LjXNuYcBhnVY5Iy3ek92zD'] []
88
{'bgZlsZBhOyfdYklWD0P1eSpnkGmHMSSBgIqg0U7y6mhrTg', 'noVL89dhsDtaHirW5gvcLXObNH86FmSW2nliCyfZrdZ0Mg', '1fuJ0O4jvzDITKYNtKSUyP4ICSz0sh2e6Q5_zmk46ms7', '55oR9A_jDNXOD8qwDS4MKqOYq5eIDELeB5_vtUQP1pI', 'N14y1D-K1ps7zFUKrz0qDQusbI7VnSWS8mIjUY_IvxFPGm4', 'X1I8Y_NvVzP-7TD76kb1QrNs1nicJ8WlA63JrU2nfHrnDg', 'oVqeijFz6WAAIzJVajjRGyjyj5flMGphTu31v4D9TZ0EYug', 'ZP-5Aobt0OAzqfBRDSmtZ76bmqG_hV5ETa-Rhmu0uNrmcf4', '3BFVM2schLDOJPlfyqyNJbUL13Lk0zoEHl0SwFYOsfchbEE', 'By8EV1-EhhhuUyIWi8omVF3LCTveLJuCzgKN2vahuDJsBfU', '7OkQaq_oasmKuMDeup2DpsC5TLUZFegUQSI4TfK6gTkU9wQ', '9_FKW82GO3Dmp5Aeg_qbLL6EAAlk6DXKY1j47lWcgweSQgo', 's-A313cl3W_jxpMqfO6PFkzQyNnNmb80pywxWDfTMUGWNQ', 'ZBo-MAJBL1gvs2qF65fXSa1yYf0h2bb-6xHI038_DlZ2MYM', 'RGvU7CjsUIEsGDO3wzMCT6sJMrEilLxqUx9d38h0ecxU', 'nMnie4tK2W8BDqYvc-PsmVP69IfrPN_feZ-gHv-0u98s6ps', 'N-rW2ppo3fcqDYaVBum0otEIbeh9oGrx0Ui_lChdrXF4NA', '5j2h-3n9CQNgb0skEfA4KfiR6TvlQvUE1Khp_QML0n3l', '3VGn8z-PtAYCfniuU9-tVj5KRKu2coaujAuW2XFSr

KeyError: 'matches'

In [10]:
len()

88

In [16]:
len(copy)

10

In [10]:
len(game_ids)

10

In [12]:
len(player_ids)

95

In [25]:
len(player_ids)

284

In [ ]:
#to do: arrumar bug!!